In [82]:
%matplotlib notebook
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [8, 8]

from faker import Factory
from itertools import islice
import geopandas as gpd
import matplotlib.pyplot as plt
import os
import pandas as pd
import pickle

import pyodbc
import pysal as ps

from pysal.viz.mapclassify import Natural_Breaks as nb

from math import log10
from shapely.geometry import Point
from shapely import wkt

from IPython.display import IFrame

import folium
from folium.plugins import HeatMap

fake = Factory.create()

from pysal.viz.mapclassify import Natural_Breaks as nb
import pandas as pd

In [62]:
data_fout = os.path.join(os.path.dirname(os.getcwd()), 'data_out')

with open(os.path.join(data_fout, 'samples_buffered_normalised_4283.pickle'), 'rb') as f:
    samples = pickle.load(f)
    
with open(os.path.join(data_fout, 'buffers_4283.pickle'), 'rb') as f:
    buffers = pickle.load(f)

In [64]:
subbuffers = buffers.cx[141:142, -31:-33]
subsamples = samples.cx[141:142, -31:-33]

In [69]:
# subbuffers.plot()
subbuffers.to_file(os.path.join(data_fout, 'subbuffers.geojson'), driver ='GeoJSON')

c:\users\gatesk\appdata\local\programs\python\python36\lib\site-packages\geopandas\io\file.py:108: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():


In [71]:
subsamples['Au_ppm_nml'].fillna(0, inplace=True)
subsamples.to_file(os.path.join(data_fout, 'subsamples'))

In [74]:
colour_spectrum =['#DEEAF6','#CFE1F3','#B1CFED','#5EC8C2','#7FBB7F','#7EC067','#E68D3D','#DE5003','#C11C17','#A6358C']
coloured = gpd.GeoDataFrame()
element = 'Au_ppm_nml'

for subset, samples in subsamples.groupby(['RIN', 'index']):
        minv = samples[element].min()
        samples[element].fillna(0, inplace=True)
        
        classifier = nb(samples['Au_ppm_nml'], 10)
        samples['nat_breaks'] = samples['Au_ppm_nml'].apply(classifier)
        
        samples.nat_breaks.replace([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], colour_spectrum, inplace=True)
        
        coloured = pd.DataFrame(pd.concat([coloured, samples], ignore_index=True))

c:\users\gatesk\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\users\gatesk\appdata\local\programs\python\python36\lib\site-packages\pandas\core\generic.py:5890: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
c:\users\gatesk\appdata\local\programs\python\python36\lib\site-packages\pysal\viz\mapclassify\classifiers.py:1428: UserWarning: Warning: Not enough unique values in array to form k classes
  Warn(ms, UserWarning)
c:\users\gatesk\appdata\local\programs\python\python36\lib\site-pac

In [81]:
# element = 'Au_ppm_fn'

# for subset, samples in subsamples.groupby(['RIN', 'index']):
#     minv = samples[element].min()
#     maxv = samples[element].max()
#     mad = samples[element].mad()
#     median = samples[element].median()
#     mean = samples[element].mean()
#     threshold = median + (mad+mad)
#     thes2 = mean + (2*mad)
#     try:
#         print(subset, median.round(3), mean.round(3), mad.round(3), threshold.round(3), thes2.round(3))
#     except:
#         pass

In [80]:
# element = 'Au_ppm_nml'
# for subset, samples in subsamples.groupby(['RIN', 'index']):
# #     if subset[0] == 'R00019368':
#         minv = samples[element].min()
#         maxv = samples[element].max()
#         mad = samples[element].mad()
#         median = samples[element].median()
#         mean = samples[element].mean()
#         threshold = median + (mad+mad)
#         thes2 = mean + (mad)
#         try:
#             print(subset, median.round(3), mean.round(3), mad.round(3), threshold.round(3), thes2.round(3))
#             samples[element].hist()
#         except:
#             pass

In [138]:
# subsamples['Au_bins'] = pd.cut(subsamples.Au_ppm_nml, froms, labels=colours, include_lowest=True)

c:\users\gatesk\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [76]:
# to geodataframe 4283
coloured = gpd.GeoDataFrame(coloured)

# Make geometry field wkt (Well Known Text)
coloured['Coordinates'] = list(zip(coloured.LNG94.round(6), coloured.LAT94.round(6))) # Round to 6dp
coloured['Coordinates'] = coloured['Coordinates'].apply(Point)
coloured = gpd.GeoDataFrame(coloured, geometry='Coordinates', crs={'init' :'epsg:4283'})

In [77]:
coloured.drop('Coordinates_4283', inplace=True, axis=1)

In [78]:
# to geojson
coloured.to_file(os.path.join(data_fout, 'coloured.geojson'), driver ='GeoJSON')

In [79]:
# to pickle
with open(os.path.join(data_fout, 'coloured.pickle'), 'wb') as f:
    pickle.dump(coloured, f, pickle.HIGHEST_PROTOCOL)